In [1]:
%load_ext autoreload
%autoreload 2

# Compute Performance Data and Save for Plotting P-Charts

In [9]:
import os
from performance import compute_and_save_performance
suffix = "_true.csv"
for entry in os.listdir("evaluation_data"):
    if suffix in entry:
        name = entry[:-len(suffix)]
        print(name)
        compute_and_save_performance(name)

characteristic_bolete.h5
====================================================================================================done


KeyboardInterrupt: 

# Get baselines for p-charts
## Random Performance by Characteristic

In [26]:
import numpy as np
from performance import estimate_performance
from data_utils import save_performance_data

density = 100
num_samples = 1000
C = 38
results = np.zeros((density,2))
p_num = 0
p_value = np.linspace(0,1,density)
for p in p_value:
    print("=", end="")
    samples = np.zeros((num_samples, 172))
    for s in range(num_samples):
        u = np.random.randint(0,2,(C,))
        samples[s,:] = estimate_performance(u, p).T
    results[p_num,:] = [p, np.mean(samples)]
    p_num += 1

save_performance_data(results[:,0], results[:,1], "baseline")

## Random Performance By Mushroom (For Direct)

In [9]:
import numpy as np
from performance import estimate_performance_direct
from data_utils import save_performance_data

density = 100
C = 38
M = 172
num_samples = M
results = np.zeros((density,2))
p_num = 0
p_value = np.linspace(0,1,density)
for p in p_value:
    print("=", end="")
    samples = np.zeros((num_samples, M))
    for s in range(num_samples):
        ids = np.random.permutation(np.arange(0,M))
        samples[s,:] = estimate_performance_direct(ids, p)
    results[p_num,:] = [p, np.mean(samples)]
    p_num += 1

save_performance_data(results[:,0], results[:,1], "random_species")

True

## Perfect Performance

In [4]:
import numpy as np
from data_utils import save_performance_data
from lookalikes import get_feats, get_num_species, get_performance

density = 100
C = 38
results = np.zeros((density+1,2))
p_num = 0
for p in range(101):
    print("=", end="")

    perf = np.zeros((get_num_species(),))

    for id in range(get_num_species()):
        u = get_feats(id)
        perf[id] = get_performance(u, id, p/100)

    results[p_num,:] = [p/100, np.mean(perf)]
    p_num += 1

save_performance_data(results[:,0], results[:,1], "perfect")

True